In [ ]:
!pip install torch torchvision torchaudio
!pip install pytorch-lightning
!pip install psutil

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
import time
import psutil
import os

In [ ]:
# Step 1: Prepare Dataset
from torchvision.datasets import VOCDetection

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = VOCDetection(root='/content/Dataset', year='2012', image_set='train', download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

test_dataset = VOCDetection(root='/content/Dataset', year='2012', image_set='val', download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Create a mapping from class names to indices
class_names = [
    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow",
    "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]
class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}
idx_to_class = {v: k for k, v in class_to_idx.items()}

100%|██████████| 1999639040/1999639040 [01:10<00:00, 28190198.72it/s]


Extracting /content/Dataset/VOCtrainval_11-May-2012.tar to /content/Dataset
Using downloaded and verified file: /content/Dataset/VOCtrainval_11-May-2012.tar
Extracting /content/Dataset/VOCtrainval_11-May-2012.tar to /content/Dataset


In [ ]:
# Function to preprocess targets
def preprocess_targets(targets, device):
    processed_targets = []
    for target in targets:
        objects = target['annotation']['object']
        if not isinstance(objects, list):
            objects = [objects]

        boxes = []
        labels = []
        for obj in objects:
            bbox = obj['bndbox']
            box = [float(bbox['xmin']), float(bbox['ymin']), float(bbox['xmax']), float(bbox['ymax'])]
            boxes.append(box)
            labels.append(class_to_idx[obj['name']])

        processed_target = {
            'boxes': torch.tensor(boxes, dtype=torch.float32).to(device),
            'labels': torch.tensor(labels, dtype=torch.int64).to(device)
        }
        processed_targets.append(processed_target)

    return processed_targets

In [ ]:
# Step 2: Define the R-FCN Model
class RFCNModel(LightningModule):
    def __init__(self):
        super(RFCNModel, self).__init__()
        self.model = fasterrcnn_resnet50_fpn(pretrained=True)
        self.model.train()

    def forward(self, images, targets=None):
        if self.training:
            return self.model(images, targets)
        else:
            return self.model(images)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        device = images[0].device
        targets = preprocess_targets(targets, device)
        loss_dict = self.model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        self.log('train_loss', losses, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return losses

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        return optimizer

In [ ]:
checkpoint_dir = '/content/models'
os.makedirs(checkpoint_dir, exist_ok=True)
print(f"Checkpoint directory: {checkpoint_dir}")

Checkpoint directory: /content/models


In [ ]:
# Step 3: Train the Model
model = RFCNModel()

checkpoint_callback = ModelCheckpoint(
    monitor='train_loss',
    dirpath=checkpoint_dir,
    filename='model-{epoch:02d}-{train_loss:.2f}',
    save_top_k=1,
    mode='min'
)
print(f"Checkpoint callback set to save in: {checkpoint_callback.dirpath}")

trainer = Trainer(
    max_epochs=5,
    devices=1 if torch.cuda.is_available() else None,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    precision=16,  # Use mixed precision training
    callbacks=[checkpoint_callback]
)

start_time = time.time()
trainer.fit(model, train_loader)
end_time = time.time()

training_time = end_time - start_time

# Force a checkpoint save after training
print(f"Forcing checkpoint save...")
trainer.save_checkpoint(os.path.join(checkpoint_dir, "final_model.ckpt"))

# Verify checkpoint creation
print(f"Checkpoint files: {os.listdir(checkpoint_dir)}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:00<00:00, 213MB/s]
/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but 

Checkpoint callback set to save in: /content/models


INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | FasterRCNN | 41.8 M | train
---------------------------------------------
41.5 M    Trainable params
222 K     Non-trainable params
41.8 M    Total params
167.021   Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottlenec

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the ba

Forcing checkpoint save...
Checkpoint files: ['final_model.ckpt', 'model-epoch=04-train_loss=0.48.ckpt']


In [ ]:
# Step 4: Calculate Misclassifications
def calculate_misclassifications(outputs, targets):
    misclassifications = 0
    for output, target in zip(outputs, targets):
        if 'boxes' not in output or len(output['boxes']) == 0:
            # If there are no predictions, consider it a misclassification
            misclassifications += 1
            continue

        # Get the predicted class with the highest confidence
        predicted_class = output['labels'][0].item() if len(output['labels']) > 0 else -1  # Assuming the highest confidence box is the first
        # Get the ground truth class
        objects = target['annotation']['object']
        if not isinstance(objects, list):
            objects = [objects]
        ground_truth_class = objects[0]['name']

        # Convert ground truth class to corresponding label index
        ground_truth_class_idx = class_to_idx.get(ground_truth_class, -1)

        # Compare predicted class with ground truth class
        if predicted_class != ground_truth_class_idx:
            misclassifications += 1

    return misclassifications

In [ ]:
def evaluate_model(model, test_loader, max_batches=10):
    model.eval()
    misclassifications = 0
    total = 0
    inference_times = []
    batch_count = 0

    for batch_idx, (images, targets) in enumerate(test_loader):
        if batch_count >= max_batches:
            break

        with torch.no_grad():
            for img, target in zip(images, targets):
                start_time = time.time()
                output = model(img.unsqueeze(0))  # Add batch dimension
                end_time = time.time()
                inference_time = end_time - start_time
                inference_times.append(inference_time)

                misclassifications += calculate_misclassifications(output, [target])
                total += 1

                # Print predictions
                print(f"Batch {batch_idx+1}, Image {total}:")
                print(f"  Inference Time: {inference_time:.6f} seconds")
                print(f"  Predictions:")
                for j, box in enumerate(output[0]['boxes']):
                    label = output[0]['labels'][j].item()
                    score = output[0]['scores'][j].item()
                    print(f"   - Label: {idx_to_class[label]}, Score: {score:.4f}, Box: {box.tolist()}")

        batch_count += 1

    avg_inference_time = sum(inference_times) / len(inference_times)
    misclassification_rate = (misclassifications / total) * 100

    return misclassification_rate, avg_inference_time

misclassification_rate, avg_inference_time = evaluate_model(model, test_loader, max_batches=10)

print(f"Misclassification Rate: {misclassification_rate:.2f}%")
print(f"Average Inference Time per Prediction: {avg_inference_time:.6f} seconds")

Batch 1, Image 1:
  Inference Time: 2.351446 seconds
  Predictions:
   - Label: tvmonitor, Score: 0.7856, Box: [69.79357147216797, 3.8188934326171875, 495.3821716308594, 325.5989074707031]
   - Label: car, Score: 0.5010, Box: [56.49803161621094, 175.7566680908203, 500.0, 351.16033935546875]
   - Label: person, Score: 0.3399, Box: [23.524595260620117, 92.61656188964844, 228.2487030029297, 360.00360107421875]
   - Label: person, Score: 0.2411, Box: [0.0, 107.33599853515625, 68.7531509399414, 374.2991943359375]
   - Label: person, Score: 0.1619, Box: [360.0212707519531, 123.45246887207031, 500.0, 366.9862060546875]
   - Label: person, Score: 0.1544, Box: [0.0, 64.03742980957031, 117.95929718017578, 368.19384765625]
   - Label: diningtable, Score: 0.1457, Box: [45.188419342041016, 85.6759033203125, 500.0, 375.0]
   - Label: sofa, Score: 0.1341, Box: [42.462921142578125, 65.06571197509766, 500.0, 369.03265380859375]
   - Label: person, Score: 0.1154, Box: [69.71154022216797, 89.984611511230

In [ ]:
# Step 6: Measure Resource Requirements and Storage Consumption
import os

# Measure Memory Usage
memory_usage = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2  # in MB

# Measure Model Storage Consumption


model_size = sum(os.path.getsize(os.path.join(checkpoint_dir, f)) for f in os.listdir(checkpoint_dir) if os.path.isfile(os.path.join(checkpoint_dir, f))) / 1024 ** 2  # in MB

print(f"Training Time: {training_time} seconds")
print(f"Misclassification Rate: {misclassification_rate}%")
print(f"Average Inference Time {avg_inference_time} seconds")
print(f"Memory Usage: {memory_usage} MB")
print(f"Model Storage Consumption: {model_size} MB")

Training Time: 4046.201509475708 seconds
Misclassification Rate: 60.0%
Average Inference Time 1.4189377814531325 seconds
Memory Usage: 5575.44921875 MB
Model Storage Consumption: 953.4432716369629 MB
